# word2vec: How To Implement word2vec

### Explore Pre-trained Embeddings

Some other options:
- `glove-twitter-{25/50/100/200}`
- `glove-wiki-gigaword-{50/200/300}`
- `word2vec-google-news-300`
- `word2vec-ruscorpora-news-300`

In [1]:
# Install gensim
# !pip install -U gensim

In [2]:
# Load pretrained word vectors using gensim
import gensim.downloader as api

wiki_embeddings = api.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [3]:
# Explore the word vector for "king"
wiki_embeddings['king']

array([-0.32307 , -0.87616 ,  0.21977 ,  0.25268 ,  0.22976 ,  0.7388  ,
       -0.37954 , -0.35307 , -0.84369 , -1.1113  , -0.30266 ,  0.33178 ,
       -0.25113 ,  0.30448 , -0.077491, -0.89815 ,  0.092496, -1.1407  ,
       -0.58324 ,  0.66869 , -0.23122 , -0.95855 ,  0.28262 , -0.078848,
        0.75315 ,  0.26584 ,  0.3422  , -0.33949 ,  0.95608 ,  0.065641,
        0.45747 ,  0.39835 ,  0.57965 ,  0.39267 , -0.21851 ,  0.58795 ,
       -0.55999 ,  0.63368 , -0.043983, -0.68731 , -0.37841 ,  0.38026 ,
        0.61641 , -0.88269 , -0.12346 , -0.37928 , -0.38318 ,  0.23868 ,
        0.6685  , -0.43321 , -0.11065 ,  0.081723,  1.1569  ,  0.78958 ,
       -0.21223 , -2.3211  , -0.67806 ,  0.44561 ,  0.65707 ,  0.1045  ,
        0.46217 ,  0.19912 ,  0.25802 ,  0.057194,  0.53443 , -0.43133 ,
       -0.34311 ,  0.59789 , -0.58417 ,  0.068995,  0.23944 , -0.85181 ,
        0.30379 , -0.34177 , -0.25746 , -0.031101, -0.16285 ,  0.45169 ,
       -0.91627 ,  0.64521 ,  0.73281 , -0.22752 , 

In [4]:
# Find the words most similar to king based on the trained word vectors
wiki_embeddings.most_similar('king') # search for word vectors and return vectors that looks similar to king

[('prince', 0.7682329416275024),
 ('queen', 0.7507690787315369),
 ('son', 0.7020888328552246),
 ('brother', 0.6985775828361511),
 ('monarch', 0.6977890729904175),
 ('throne', 0.691999077796936),
 ('kingdom', 0.6811409592628479),
 ('father', 0.6802029013633728),
 ('emperor', 0.6712858080863953),
 ('ii', 0.6676074266433716)]

### Train Our Own Model

In [5]:
# Read in the data and clean up column names
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('../../../data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]
messages.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [8]:
# Clean data using the built in cleaner in gensim
messages['text_clean'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x)) # used gensim to do the cleaning
messages.head()

,label,text,text_clean
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g...","[go, until, jurong, point, crazy, available, only, in, bugis, great, world, la, buffet, cine, th..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,"[free, entry, in, wkly, comp, to, win, fa, cup, final, tkts, st, may, text, fa, to, to, receive,..."
3,ham,U dun say so early hor... U c already then say...,"[dun, say, so, early, hor, already, then, say]"
4,ham,"Nah I don't think he goes to usf, he lives around here though","[nah, don, think, he, goes, to, usf, he, lives, around, here, though]"


In [9]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

In [11]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   #size=100,
                                   vector_size=100,
                                   window=5, # before and after the word
                                   min_count=2 # number of the times the word should appear in the corpus
                                  )

In [12]:
# Explore the word vector for "king" base on our trained model
w2v_model.wv['king']

array([-0.05833323,  0.07032395,  0.01269502,  0.01098211,  0.02242075,
       -0.11713988,  0.04783008,  0.18817279, -0.06652589, -0.03635075,
       -0.05016559, -0.1279992 , -0.00828839,  0.02731685,  0.05207808,
       -0.08674876,  0.05074142, -0.10147814, -0.01908107, -0.16013202,
        0.04248298,  0.02050967,  0.05710406, -0.03096555, -0.01793025,
        0.01499313, -0.07335565, -0.08032767, -0.07436841,  0.01868421,
        0.07173979, -0.00773722,  0.05348251, -0.04383763, -0.03016734,
        0.09790911,  0.00619834, -0.06877115, -0.05622029, -0.12127917,
       -0.00680044, -0.07088436, -0.05287399,  0.04172011,  0.0888371 ,
       -0.05358858, -0.03828558, -0.00387682,  0.02531683,  0.06547289,
        0.02087918, -0.0947846 ,  0.00131498, -0.00883692, -0.02752241,
        0.0595649 ,  0.0296939 ,  0.00508708, -0.0662582 ,  0.04203625,
        0.03048635,  0.03221356,  0.01226585, -0.00911734, -0.09567153,
        0.08073927,  0.03588324,  0.0663904 , -0.12833516,  0.11

In [14]:
# Find the most similar words to "king" based on word vectors from our trained model
w2v_model.wv.most_similar('king')

[('miss', 0.9967660307884216),
 ('box', 0.9967115521430969),
 ('show', 0.9966834187507629),
 ('any', 0.9966814517974854),
 ('been', 0.9966760277748108),
 ('as', 0.9966288805007935),
 ('an', 0.9966248869895935),
 ('st', 0.9966089725494385),
 ('should', 0.9965878129005432),
 ('great', 0.9965769648551941)]